In [27]:
# Descarga tu repositorio desde GitHub
!git clone https://github.com/Wmarbar/CProgram_incidencias.git

# Cambia de directorio para acceder a los archivos
%cd CProgram_incidencias

# Verifica que los archivos estén
!ls


Cloning into 'CProgram_incidencias'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 45 (delta 22), reused 9 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 1.64 MiB | 16.26 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias/CProgram_incidencias
 constructor4_proyecto_II.ipynb   README.md
 input				 'Trabajo-CalidadDeDatos Ver4-2.pdf'
 logo_u.png


# __PROGRAMACIÓN PARA ANALÍTICA DE DATOS__

<img src="https://github.com/WMARBAR/CProgram_incidencias/blob/main/logo_u.png?raw=1" alt="Logo Universidad Central" width="300" />

> ## __PROYECTO: Estimación Área bajo la curva__

> ## Profesor: Jorge Victorino [jvictorinog@ucentral.edu.co](jvictorinog@ucentral.edu.co)<br>
> ## Estudiante: Sergio Andrés Sánchez Cárdenas [correo@ucentral.edu.co](mailto:correo@ucentral.edu.co)<br>
> ## Estudiante: Wilson Felipe Mártinez Barrantes [wmartinezb1@ucentral.edu.co](jvictorinog@ucentral.edu.co)<br>
> ## Grupo: 5 <br>
> ### Facultad de Ingeniería y Ciencias Básicas <br>
> ### Universidad Central <br>
> <br>

---


# FUNCTIONS

In [28]:
import pandas as pd
import numpy as np

def load_excel(path):

 df_incidentes = pd.read_excel(path)
 return df_incidentes



def delta_dates(df, col1, col2):
    try:
        # Verificar si las columnas existen
        if col1 not in df.columns or col2 not in df.columns:
            print(f"Las columnas '{col1}' o '{col2}' no existen en el DataFrame.")
            return df

        # Convertir columnas a datetime con coerción
        df[col1] = pd.to_datetime(df[col1], errors='coerce')
        df[col2] = pd.to_datetime(df[col2], errors='coerce')

        # Avisar si hay muchas fechas inválidas (pero continuar)
        if df[col1].isna().any():
            print(f"Algunos valores en '{col1}' no son fechas válidas y se marcarán como NaT.")
        if df[col2].isna().any():
            print(f"Algunos valores en '{col2}' no son fechas válidas y se marcarán como NaT.")

        # Calcular diferencia en días (donde ambas fechas sean válidas)
        nueva_col = f"{col1}_{col2}_days"
        df[nueva_col] = (df[col2] - df[col1]).dt.days

        return df

    except Exception as e:
        print(f"Error inesperado: {e}")
        return df

def key_generator(df, columnas):
    try:
        # Validar columnas
        for col in columnas:
            if col not in df.columns:
                print(f"La columna '{col}' no existe en el DataFrame.")
                return df

        # Nombre de la nueva columna
        nombre_key = '_'.join(columnas) + '_KEY'

        # Función de limpieza para cada valor
        def limpiar_valor(v):
            if isinstance(v, float) and v.is_integer():
                return str(int(v))
            return str(v)

        # Aplicar limpieza y concatenar
        df[nombre_key] = df[columnas].apply(lambda row: '_'.join(limpiar_valor(v) for v in row), axis=1)

        return df

    except Exception as e:
        print(f"Error inesperado: {e}")
        return df


def drop_nan_by_column(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return df

    df_limpio = df.dropna(subset=[col])
    return df_limpio


def deduplicar_incidentes(df, key_col='Incidente_Estructura_KEY', dias_col='Creacion_Inicio_days'):
    # Verificar columnas
    if key_col not in df.columns or dias_col not in df.columns:
        print(f"Las columnas '{key_col}' o '{dias_col}' no existen.")
        return df

    # Función para aplicar a cada grupo
    def resolver_grupo(grupo):
        # Filtrar los que no tienen 0
        sin_cero = grupo[grupo[dias_col] != 0]
        if not sin_cero.empty:
            return sin_cero.iloc[[0]]  # Quedarse con uno cualquiera de los buenos
        else:
            return grupo.iloc[[0]]  # Si todos son 0, igual dejar uno

    # Agrupar por la clave y aplicar la lógica
    df_filtrado = df.groupby(key_col, group_keys=False).apply(resolver_grupo).reset_index(drop=True)

    return df_filtrado

def unique_validator(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return

    duplicados = df.duplicated(subset=[col]).sum()

    if duplicados > 0:
        print(f"Hay {duplicados} valores duplicados en la columna '{col}'.")
    else:
        print(f"Todos los valores en la columna '{col}' son únicos.")



def nan_detector_column(df, col):
    if col not in df.columns:
        print(f"La columna '{col}' no existe en el DataFrame.")
        return df

    df[f'{col}_NAN_DETECTOR'] = df[col].apply(lambda x: 1 if pd.isna(x) or (isinstance(x, str) and x.strip() == '') else 0)

    return df

def deduplicar_estructuras_por_vereda(df, key_col='Estructura', prioridad_col='Vereda'):
    # Validar que las columnas existan
    if key_col not in df.columns or prioridad_col not in df.columns:
        print(f"❌ Las columnas '{key_col}' o '{prioridad_col}' no existen.")
        return df

    # Función para aplicar por grupo
    def resolver_grupo(grupo):
        con_vereda = grupo[grupo[prioridad_col].apply(lambda x: isinstance(x, str) and x.strip() != '') | grupo[prioridad_col].notna()]
        if not con_vereda.empty:
            return con_vereda.iloc[[0]]  # Uno cualquiera con vereda
        else:
            return grupo.iloc[[0]]  # Si no hay vereda, igual uno cualquiera

    # Aplicar la lógica por grupo
    df_filtrado = df.groupby(key_col, group_keys=False).apply(resolver_grupo).reset_index(drop=True)

    return df_filtrado


def limpiar_nombres_columnas(df):
    df.columns = df.columns.str.strip().str.replace(r'\s+', '_', regex=True)
    return df

def corregir_basura_columna(df, col):
    if col not in df.columns:
        print(f"❌ La columna '{col}' no existe en el DataFrame.")
        return df

    def _fix_encoding(x):
        if isinstance(x, str):
            try:
                return x.encode('latin1').decode('utf-8')
            except Exception:
                return x
        return x

    df[col] = df[col].apply(_fix_encoding)
    return df

def merge_excluir_llave(df1, df2, llave_df1, llave_df2):
    # Verificar existencia de las columnas
    if llave_df1 not in df1.columns or llave_df2 not in df2.columns:
        print(f"❌ Las llaves '{llave_df1}' o '{llave_df2}' no existen.")
        return df1

    # Hacer el merge completo primero
    df_merged = df1.merge(df2, left_on=llave_df1, right_on=llave_df2, how='left')

    # Eliminar la llave de df2 si es distinta a la de df1
    if llave_df1 != llave_df2:
        df_merged = df_merged.drop(columns=[llave_df2])

    return df_merged

def guardar_excel(df, nombre_archivo='salida.xlsx', sheet_name='Hoja1'):
    try:
        df.to_excel(nombre_archivo, index=False, sheet_name=sheet_name)
        print(f"✅ Archivo guardado exitosamente como '{nombre_archivo}'.")
    except Exception as e:
        print(f"❌ Error al guardar el archivo: {e}")





# CARGA DE ARCHIVOS

In [29]:

df_incidentes = load_excel("./input/incidentes1.xlsx")
df_estructura = load_excel("./input/estructura1.xlsx")

# DEV ZONE INCIDENCIAS

## ARREGLO DE LA DATA INCIDENCIAS

In [30]:
df_incidentes.columns

Index(['Incidente', 'Estructura', 'Tipo incidente', 'Estado', 'Prioridad',
       'Creacion', 'Inicio', 'Cierre'],
      dtype='object')

In [31]:
df_incidentes=delta_dates(df_incidentes,'Creacion','Inicio')
df_incidentes=delta_dates(df_incidentes,'Creacion','Cierre')
df_incidentes=delta_dates(df_incidentes,'Inicio','Cierre')
df_incidentes=key_generator(df_incidentes,['Incidente','Estructura'])
df_incidentes=drop_nan_by_column(df_incidentes,'Estructura')
df_incidentes=deduplicar_incidentes(df_incidentes)
df_incidentes=nan_detector_column(df_incidentes,'Cierre')
df_incidentes=limpiar_nombres_columnas(df_incidentes)

Algunos valores en 'Cierre' no son fechas válidas y se marcarán como NaT.
Algunos valores en 'Cierre' no son fechas válidas y se marcarán como NaT.


<ipython-input-28-06e7bb11602c>:90: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtrado = df.groupby(key_col, group_keys=False).apply(resolver_grupo).reset_index(drop=True)


# DEV ZONE Estructura

## ARREGLO DE LA DATA ESTRUCTURAS

In [32]:

df_estructura= limpiar_nombres_columnas(df_estructura)
df_estructura= drop_nan_by_column(df_estructura,'Estructura')
df_estructura= deduplicar_estructuras_por_vereda(df_estructura)
df_estructura = corregir_basura_columna(df_estructura, 'Depto')
df_estructura['Depto'] = df_estructura['Depto'].replace('NARIÃ‘O', 'NARIÑO')
df_estructura = corregir_basura_columna(df_estructura, 'Municipio')
df_estructura['Municipio'] = df_estructura['Municipio'].replace('IBAGUÃ‰', 'IBAGUÉ')
df_estructura['Municipio'] = df_estructura['Municipio'].replace('JERUSALÃ‰N', 'JERUSALÉN')
df_estructura['Municipio'] = df_estructura['Municipio'].replace('GUACHENÃ‰', 'GUACHENÉ')
df_estructura['Latitud'] = df_estructura['Latitud'].replace(',', '.')
df_estructura['Longitud'] = df_estructura['Longitud'].replace(',', '.')

<ipython-input-28-06e7bb11602c>:132: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtrado = df.groupby(key_col, group_keys=False).apply(resolver_grupo).reset_index(drop=True)


In [33]:
df_incidentes.columns

Index(['Incidente', 'Estructura', 'Tipo_incidente', 'Estado', 'Prioridad',
       'Creacion', 'Inicio', 'Cierre', 'Creacion_Inicio_days',
       'Creacion_Cierre_days', 'Inicio_Cierre_days',
       'Incidente_Estructura_KEY', 'Cierre_NAN_DETECTOR'],
      dtype='object')

# DEV ZONE Base de incidencias completa

In [34]:
df_merged= merge_excluir_llave(df_incidentes,df_estructura,'Estructura','Estructura')


In [35]:
df_merged

,Incidente,Estructura,Tipo_incidente,Estado,Prioridad,Creacion,Inicio,Cierre,Creacion_Inicio_days,Creacion_Cierre_days,Inicio_Cierre_days,Incidente_Estructura_KEY,Cierre_NAN_DETECTOR,Serie,Depto,Municipio,Vereda,Cia,Latitud,Longitud
0,1012403,19798886.0,NaN,INIC ENPT,Semestre,2017-05-29,2017-05-29,2017-10-18,0,142.0,142.0,1012403_19798886,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1012554,19878537.0,NaN,INIC ENPT,Dos años,2017-05-28,2018-12-19,2018-12-21,570,572.0,2.0,1012554_19878537,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1012557,19877964.0,NaN,INIC ENPT,Dos años,2017-05-28,2018-06-23,2018-07-07,391,405.0,14.0,1012557_19877964,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1012558,19877965.0,NaN,INIC ENPT,Dos años,2017-05-28,2018-12-27,2018-12-27,578,578.0,0.0,1012558_19877965,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1012571,19876291.0,NaN,INIC ENPT,Dos años,2017-05-28,2018-08-08,2018-08-08,437,437.0,0.0,1012571_19876291,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30885,2999551,19862596.0,NaN,INIC ENPT,NaN,2013-07-28,2013-07-28,2014-05-11,0,287.0,287.0,2999551_19862596,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30886,2999591,19892475.0,NaN,INIC,Año,2013-07-28,2013-07-28,2013-08-01,0,4.0,4.0,2999591_19892475,0,2121.0,BOYACÁ,SANTA MARÍA,CULIMA,Zen,4.796073,-73.290555
30887,2999928,19986491.0,NaN,INIC,Semestre,2013-07-27,2013-07-27,2013-09-08,0,43.0,43.0,2999928_19986491,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30888,2999957,19862048.0,NaN,INIC ENPT,Semestre,2013-07-27,2013-07-27,2013-11-18,0,114.0,114.0,2999957_19862048,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
guardar_excel(df_merged)

✅ Archivo guardado exitosamente como 'salida.xlsx'.
